Autor: Antunes Dantas da Silva

A ideia deste lab é criar uma aplicação onde vamos passar o id de um usuário e o sistema retornará quais os filmes indicados para ele. O código da aplicação está abaixo, é bem simples. Foi utilizado o Surprise como framework de criação do sistema de recomendação. A idéia foi treinar com as notas dadas pelos usuários e sugerir filmes para um usuário baseado na escolha de usuários parecidos com isso.

Para isso, utilizamos o KNN como algoritmo de predição. O RMSE da predição pode ser visto no final do bloco de código.

Para testar a aplicação, basta entrar no link a seguir que contém um exemplo onde você diz um usuário e o sistema diz quais filmes são indicados para ele.

In [2]:
from surprise import KNNBasic, Reader, accuracy
from surprise.model_selection import PredefinedKFold
from surprise import Dataset
import pandas as pd
import os
import csv

# Primeiro, vamos ler os arquivos.

items_stream = open('./static/ml-100k/u.item', 'r', encoding="ISO-8859-1")
item_data = items_stream.read().split('\n')
item_data = list(map(lambda item: item.split('|')[:2], item_data))
items_stream.close()

# Depois, vamos criar a coleção de ites e de usuários, além de um dicionário contendo os filmes que
# os usuários não assistiram.
database = pd.read_csv('./static/ml-100k/u1.base.csv')
usuarios = set(database.user_id)
filmes = set(database.item_id)
filmes_nao_assistidos = {usuario: filmes.difference(database.query('user_id == %s' % (usuario)).item_id) for usuario in usuarios}

files_dir = os.path.expanduser('./static/ml-100k/')

# Para a geração do recomendados, vamos utilizar os dados built in na biblioteca
reader = Reader('ml-100k')

# Dividindo os dados em pastas diferentes para uma melhor prediçao
train_file = files_dir + 'u%d.base'
test_file = files_dir + 'u%d.test'
folds_files = [(train_file % i, test_file % i) for i in [1]]

data = Dataset.load_from_folds(folds_files, reader=reader)
pkf = PredefinedKFold()

sim_options = {
    'name': 'cosine', # distancia do cosseno
    'user_based': True  # similiridade entre os usuarios
}

algo = KNNBasic(sim_options=sim_options)

for trainset, testset in pkf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions_knn = algo.test(testset)
    rmse_knn = accuracy.rmse(predictions_knn)


mapa_de_filmes = {}

with open("./static/ml-100k/u.item", encoding="ISO-8859-1") as tsv:
    for line in csv.reader(tsv, dialect="excel-tab"):
        linha = line[0].split('|')
        mapa_de_filmes[linha[0]] = linha[1]

def get_top_5_movies(userId):
    filmes = filmes_nao_assistidos[int(userId)]
    predicoes = [(filme, algo.predict(uid=userId, iid=str(filme)).est) for filme in filmes]
    top_5 = sorted(predicoes, key=lambda item: item[1], reverse=True)[:5]
    top_5 = [mapa_de_filmes[str(filme[0])] for filme in top_5]

    return top_5

def get_k_nearest_neighbors(userId):
    # Retrieve inner id of the user
    inner_id = algo.trainset.to_inner_uid(userId)

    # Retrieve inner ids of the nearest neighbors of the requested user
    user_neighbors = algo.get_neighbors(inner_id, k=3)

    # Convert inner ids to raw ids
    neighbors = (algo.trainset.to_raw_uid(inner_id)
                           for inner_id in user_neighbors)
    neigbhors_list = [id for id in neighbors]
    return neigbhors_list

def getRecommendations(uid):
    movies = get_top_5_movies(uid)
    neighbours = get_k_nearest_neighbors(str(uid))
    return {
        'movies': movies,
        'neighbours': neighbours
    }

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0229
